In [259]:
import pandas as pd
import glob
import os 
import re
import itertools

In [3]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', 60)

In [5]:
cwd = os.getcwd()
events = []
os.chdir(cwd + '\\notes')
events = []
#convert all txt files in cwd to df, append to events, then concatenate
for file in glob.glob("*.txt"):
    data = pd.read_csv(file, encoding ='latin1')
    events.append(data)
events = pd.concat(events)
events = events.reset_index()

In [6]:
events.sample(3)

,index,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ELEMID,CHARTTIME,REALTIME,CGID,CORRECTION,CUID,CATEGORY,TITLE,TEXT,EXAM_NAME,PATIENT_INFO
3214,18,8481,13331.0,10542.0,NaN,3298-12-21 00:00:00 EST,NaN,NaN,NaN,NaN,RADIOLOGY_REPORT,NaN,\r\n\r\n\r\n DATE: [**3298-12-21**] 12:20 PM\r\n ...,NaN,NaN
4965,35,8835,3119.0,10989.0,0.0,3116-04-03 05:21:00 EST,3116-04-03 05:32:00 EST,3601.0,N,54.0,Nursing/Other,NaN,\r\n\r\nPersistent [**Female First Name (un) **]\r\n\r\n...,NaN,NaN
2456,146,8370,4174.0,10407.0,0.0,3108-06-02 14:52:00 EST,3108-06-02 14:58:00 EST,2788.0,N,69.0,Nursing/Other,NaN,\r\n\r\ntransfered to the unit with septic shock and pna...,NaN,NaN


In [7]:
events.columns

Index(['index', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ELEMID', 'CHARTTIME',
       'REALTIME', 'CGID', 'CORRECTION', 'CUID', 'CATEGORY', 'TITLE', 'TEXT',
       'EXAM_NAME', 'PATIENT_INFO'],
      dtype='object')

In [8]:
discharge = events.loc[events.CATEGORY == 'DISCHARGE_SUMMARY'].TEXT.tolist()
for i in range(len(discharge)):
    discharge[i] = discharge[i].replace('\n','newline').replace('\r',' ')

In [9]:
diagnosis_raw = []
for i in discharge:
    if re.search(r'DISCHARGE DIAGNOSES(.*?)DISCHARGE CONDITION', i, re.IGNORECASE):
        diagnosis_raw.append(re.search(r'DISCHARGE DIAGNOSES(.*?)DISCHARGE CONDITION', i, re.IGNORECASE).group(1))
    if re.search(r'DISCHARGE DIAGNOSiS(.*?)DISCHARGE CONDITION', i, re.IGNORECASE):
        diagnosis_raw.append(re.search(r'DISCHARGE DIAGNOSiS(.*?)DISCHARGE CONDITION', i, re.IGNORECASE).group(1))

In [526]:
diagnosis = []
for j in diagnosis_raw:   
    start = []
    end = []
    for m in re.finditer('newline', j):
        start.append(m.start())
        end.append(m.end())
        
    diagnosis_ind = []
    for i in range(len(start)-1):
        diagnosis_ind.append((j[end[i]:start[i+1]]))
        
    diagnosis.append(diagnosis_ind)

#some issue I can't figure out, should be a simpler way to do the previous cell
#but skips every other new line :(
#not_working = re.findall(r'newline(.*?)newline', diagnosis_raw[1])

#some lists start with numbers. remove those numbers
for i in range(len(diagnosis)):
    for j in range(len(diagnosis[i])):
        diagnosis[i][j] = diagnosis[i][j].replace('1. ','').replace('2. ','').replace('3. ','').replace('4. ','')\
        .replace('5. ','').replace('6. ','').replace('7. ','').replace('8. ','').replace('9. ','')\
        .replace('10. ','').replace('11. ','')

#turn everything into lowercase for uniformity
for i in range(len(diagnosis)):
    diagnosis[i] = [x.lower() for x in diagnosis[i]]
#filter out blank values
for i in range(len(diagnosis)):
    diagnosis[i] = list(filter(None, diagnosis[i]))   

In [527]:
#remove values from list unrelated to a diagnosis
for i in range(len(diagnosis)):
    for j in range(len(diagnosis[i])):
        diagnosis[i][j] = diagnosis[i][j].rstrip()
    if ' ' in diagnosis[i]:
            diagnosis[i].remove(' ')  
    if '' in diagnosis[i]:
        diagnosis[i].remove('')
    if '  ' in diagnosis[i]:
        diagnosis[i].remove('  ') 
    if '   ' in diagnosis[i]:
        diagnosis[i].remove('   ')  

In [528]:
#if a value ends in punctuation or an extra space, remove that part
for i in range(len(diagnosis)):
    for j in range(len(diagnosis[i])):
        if diagnosis[i][j].endswith('.'):
            diagnosis[i][j] = diagnosis[i][j][:-1]
        if diagnosis[i][j].endswith(':'):
            diagnosis[i][j] = diagnosis[i][j][:-1]
        if diagnosis[i][j].startswith(' '):
            diagnosis[i][j] = diagnosis[i][j][1:]
#if a value starts with primary/secondary stuff, remove that part           
        if diagnosis[i][j].startswith('primary: '):
            diagnosis[i][j] = diagnosis[i][j][9:]
        if diagnosis[i][j].startswith('primary diagnosis: '):
            diagnosis[i][j] = diagnosis[i][j][19:]
        if diagnosis[i][j].startswith('secondary diagnosis: '):
            diagnosis[i][j] = diagnosis[i][j][21:]
        if diagnosis[i][j].startswith('secondary: '):
            diagnosis[i][j] = diagnosis[i][j][11:]
#also want to get rid of status bit if diagnosis starts with that
        if diagnosis[i][j].startswith('status '):
            diagnosis[i][j] = diagnosis[i][j][7:]
#gi = gastrointestinal, switch it to that
        if 'gi ' in diagnosis[i][j]:
            diagnosis[i][j] = diagnosis[i][j].replace('gi ', 'gastrointestinal ')

    

In [530]:
for i in range(len(diagnosis)):
    diagnosis[i] = list(filter(lambda k:'secondary diagnoses' not in k, diagnosis[i]))
    diagnosis[i]= list(filter(lambda k:'secondary diagnosis' not in k, diagnosis[i]))
    diagnosis[i] = list(filter(lambda k: 'secondary' not in k, diagnosis[i]))
    diagnosis[i] = list(filter(lambda k: 'primary' not in k, diagnosis[i]))
    diagnosis[i] = list(filter(lambda k: 'primary diagnosis' not in k, diagnosis[i]))
    diagnosis[i] = list(filter(lambda k: 'primary diagnoses' not in k, diagnosis[i]))

In [531]:
#when there is a comma in a diagnosis, it is actually multiple diagnoses. 
#split the string, replace the value with the first split value, then
#append the following values to the list
for i in range(len(diagnosis)):
    for j in range(len(diagnosis[i])):
        if "," in diagnosis[i][j]:
            x = diagnosis[i][j].split(", ")
            diagnosis[i][j] = x[0]
            for k in range(1, len(x)):
                diagnosis[i].append(x[k])              

In [532]:
#looked up each term shown, verified they were all the same and in the following cell changed values for uniformity
for i in range(len(diagnosis)):
    for j in range(len(diagnosis[i])):
        if "pneu" in diagnosis[i][j]:
            diagnosis[i][j] = "pneumonia"
        if "heart" in diagnosis[i][j]:
            diagnosis[i][j] = "heart failure"
        if "demen" in diagnosis[i][j]:
            diagnosis[i][j] = "dementia"
        if "del" in diagnosis[i][j]:
            diagnosis[i][j] = "delirium"
        if "renal fail" in diagnosis[i][j]:
            diagnosis[i][j] = "renal failure"
        if "renal disease" in diagnosis[i][j]:
            diagnosis[i][j] = "renal disease"
        if "diab" in diagnosis[i][j]:
            diagnosis[i][j] = "diabetes"
        if "hemor" in diagnosis[i][j]:
            diagnosis[i][j] = "hemorrhage"
        if "copd" in diagnosis[i][j]:
            diagnosis[i][j] = "chronic obstructive pulmonary disease"
        if "cancer" in diagnosis[i][j]:
            diagnosis[i][j] = "cancer"
        if "pancre" in diagnosis[i][j]:
            diagnosis[i][j] = "pancreatitis"
        if "gastroin" in diagnosis[i][j]:
            diagnosis[i][j] = "gastrointestinal bleed"
        if "stroke" in diagnosis[i][j]:
            diagnosis[i][j] = "stroke"
        if "expi" in diagnosis[i][j]:
            diagnosis[i][j] = "expired"
        if "neuro" in diagnosis[i][j]:
            diagnosis[i][j] = "neuropathy"
        if "fib" in diagnosis[i][j]:
            diagnosis[i][j] = "atrial fibrillation"
        if "depression" in diagnosis[i][j]:
            diagnosis[i][j] = "depression"
        if "hypert" in diagnosis[i][j]:
            diagnosis[i][j] = "hypertension"
        if "htn" in diagnosis[i][j]:
            diagnosis[i][j] = "hypertension"
        if "chol" in diagnosis[i][j]:
            diagnosis[i][j] = "hypercholesterolemia"
        
        

In [533]:

for i in range(len(diagnosis)):
    for j in range(len(diagnosis[i])):
        if "second" in diagnosis[i][j]:
            print(i)
            print(j)
            print(diagnosis[i][j])

In [537]:
for i in range(len(diagnosis)):
    diagnosis[i] = list(filter(None, diagnosis[i]))   

In [538]:
#needs further cleaning
diagnosis

[['left foot gangrene',
  'urosepsis',
  'atrial fibrillation',
  'esrd/hd',
  'diabetes',
  'mrsa',
  'depression',
  'glaucoma'],
 ['mrsa bacteremia',
  'pseudomonal lung infection',
  'cancer',
  'neuropathy',
  'chronic obstructive pulmonary disease',
  'atrial fibrillation',
  'hypertension',
  'hypothyroidism',
  'anemia',
  'cancer',
  'neuropathy'],
 ['death'],
 ['atrial fibrillation'],
 ['thoracic aortic aneurysm - s/p endovascular stent',
  'pneumonia',
  'emphysema',
  'lung resection',
  'nephrectomy',
  'postop',
  'chronic obstructive pulmonary disease,',
  'history of asbestosis versus mesotheilioma - s/p rul',
  'hypertension',
  ' renal cell carcinoma - s/p',
  'depression'],
 ['pancreatitis',
  'anoxic brain injury',
  'brain death',
  'pneumonia',
  'diabetes',
  'hypertension',
  'history of coronary artery disease',
  'blood loss anemia',
  'respiratory failure'],
 ['expired'],
 ['cancer'],
 ['coronary artery disease',
  'myocardial infarction',
  'depression',
  '

In [539]:
diagnosis_all = list(itertools.chain.from_iterable(diagnosis))

In [540]:
from collections import Counter
count = Counter(diagnosis_all)
count.most_common(10)

[('hypertension', 21),
 ('pneumonia', 15),
 ('atrial fibrillation', 11),
 ('cancer', 10),
 ('heart failure', 10),
 ('diabetes', 9),
 ('gastrointestinal bleed', 8),
 ('hemorrhage', 8),
 ('depression', 7),
 ('chronic obstructive pulmonary disease', 7)]

In [247]:
#changing some diagnoses to just 'gastrointestinal bleed'
diagnosis[20][0] = 'gastrointestinal bleed'
diagnosis[27][0] = 'gastrointestinal bleed'
#diagnosis[35] is due to gastric varices, append that
diagnosis[35][0] = 'gastrointestinal bleed'
diagnosis[35].append('gastric varices')
#diagnosis[53] is due to dieulafoy lesion, append that
diagnosis[53][0] = 'gastrointestinal bleed'
diagnosis[53].append('dieulafoy lesion')

#diagnoses including 'secondary to' include two issues, so I will separate them
diagnosis[27][1] = 'hemorrhagic shock'
diagnosis[27][2] = 'cardiac demand ischemia'
diagnosis[50][0] = 'delirium'
diagnosis[50].append('hyponatremia')
diagnosis[50].append('overdiuresis')
diagnosis[50][11] = 'hypotension'
diagnosis[28][7] = 'chemical pancreatitis'
diagnosis[28][8] = 'transaminitis'
diagnosis[28].append('hepatitis')
diagnosis[28][9] = 'thrombocytosis'
diagnosis[41][0] = 'gastrointestinal bleed'
diagnosis[41][1] = 'manual disimpaction'

#changing specific diagnoses to cut out unnecessary words

#'status post ep study without ablation'
diagnosis[60][1] = "post electrophysiology study"

#diagnosis[20] needs a bunch of edits:
#'lower gastrointestinal bleed secondary to cecal'
#turp = transurethral resection of the prostate, change it to full name
diagnosis[20][7] = 'prostate cancer'
diagnosis[20].append('transurethral resection of the prostate')


#'polypectomy, status post colonoscopy and'
